# Extract data and create trade database

This notebook builds a unified export database by combining U.S. state-level data (from the Census Bureau) with international trade data (from the BACI dataset by CEPII). The goal is to create a harmonized base for later Economic Complexity calculations.

In [ ]:
import requests
import time
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format) 
API_KEY = "5f3f444cd4ce64996377d78bf3310f73b4f88945"
DATASETS_DIR = './datasets/'

## 1. Downloading Trade Data
Access and extract export data by state and HS code using the Census Bureau's API. Data can be downloaded for any selected year.

In [ ]:
BASE_URL = "https://api.census.gov/data/timeseries/intltrade/exports/statehs"

def census_session():
    s = requests.Session()
    retries = Retry(
        total=5,
        backoff_factor=0.5,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"]
    )
    s.mount("https://", HTTPAdapter(max_retries=retries, pool_maxsize=20))
    return s

def fetch_month(year: int, month: int) -> pd.DataFrame:
    """Fetch our data per month."""
    params = {
        "get": "E_COMMODITY,E_COMMODITY_LDESC,COMM_LVL,YEAR,MONTH,STATE,ALL_VAL_MO",
        "YEAR": str(year),
        "MONTH": f"{int(month):02d}",
        "COMM_LVL": "HS6",
        "key": API_KEY,
    }
   
    headers = {
        "Connection": "close",
        "Accept-Encoding": "identity",
        "User-Agent": "requests (data job)"
    }

    with census_session().get(BASE_URL, params=params, headers=headers, timeout=(5, 300)) as r:
        r.raise_for_status()
        if "application/json" not in r.headers.get("Content-Type", ""):
            raise ValueError(f"Non-JSON response for {year}-{month}: {r.text[:200]}")
        data = r.json()
        df = pd.DataFrame(data[1:], columns=data[0])
        df["YEAR"] = df["YEAR"].astype(int)
        df["MONTH"] = df["MONTH"].astype(int)
        df["ALL_VAL_MO"] = pd.to_numeric(df["ALL_VAL_MO"], errors="coerce")
        return df

def fetch_year(year: int, sleep_s: float = 0.2) -> pd.DataFrame:
    dfs = []
    for m in range(1, 13):
        df = fetch_month(year, m)
        dfs.append(df)
        time.sleep(sleep_s)  # be nice to the API
    out = pd.concat(dfs, ignore_index=True)
    out = out.drop_duplicates()
    return out

In [5]:
# A) All states for all months in 2023, HS6
df_2023_all_states = fetch_year(2023)
print(df_2023_all_states.shape)
df_2023_all_states.head(2)

(1599647, 10)


,E_COMMODITY,E_COMMODITY_LDESC,COMM_LVL,YEAR,MONTH,STATE,ALL_VAL_MO,YEAR,MONTH,COMM_LVL
0,010121,"HORSES, LIVE, PUREBRED BREEDING",HS6,2023,1,-,4977511,2023,1,HS6
1,010129,"HORSES, LIVE, OTHER THAN PUREBRED BREEDING",HS6,2023,1,-,4490670,2023,1,HS6


In [6]:
# Some columns are duplicated, let's delete them
duplicate_cols_mask = df_2023_all_states.columns.duplicated()
duplicate_cols_mask

array([False, False, False, False, False, False, False,  True,  True,
        True])

In [7]:
df = df_2023_all_states.loc[:, ~duplicate_cols_mask]

In [8]:
# Since the states are identified by their abbreviations, we are reading the state reference documentation to match names.

url = "https://www2.census.gov/geo/docs/reference/state.txt"
df_states = pd.read_csv(url, delimiter='|')
df_states.columns = ['STATE_FIPS', 'USPS', 'STATE_NAME', 'STATENS']
df_states['STATE_FIPS'] = df_states['STATE_FIPS'].astype(str).str.zfill(2)
df_states = df_states[0:51] #Delete all the states that are not US territory: American Samoa, Guam, Northern Mariana Islands, Puerto Rico, U.S. Minor Outlying Islands, U.S. Virgin Islands 
df_states

,STATE_FIPS,USPS,STATE_NAME,STATENS
0,01,AL,Alabama,1779775
1,02,AK,Alaska,1785533
2,04,AZ,Arizona,1779777
3,05,AR,Arkansas,68085
4,06,CA,California,1779778
5,08,CO,Colorado,1779779
6,09,CT,Connecticut,1779780
7,10,DE,Delaware,1779781
8,11,DC,District of Columbia,1702382
9,12,FL,Florida,294478


In [9]:
# To avoid overlapping the country Georgia later, we'll consider adding St to Georgia.
df_states.loc[df_states.STATE_NAME == "Georgia", "STATE_NAME"] = "Georgia (st)"

In [10]:
# Merging both dataframes to have complete names
df_usa = df.merge(df_states, how="inner", left_on="STATE", right_on="USPS")

In [11]:
# Save df in format long to parquet
df_usa.to_parquet(DATASETS_DIR + 'df_trade_usa_states.parquet',engine='fastparquet')

In [12]:
# Group by year to delete months
df_usa_grouped = df_usa.groupby(["E_COMMODITY","STATE_NAME"])["ALL_VAL_MO"].sum().to_frame().reset_index()
df_usa_grouped.head()

,E_COMMODITY,STATE_NAME,ALL_VAL_MO
0,010121,Alabama,5440
1,010121,Arizona,186551
2,010121,Arkansas,4583
3,010121,California,21298413
4,010121,Colorado,66820


In [13]:
# Homogenize names to merge later with the global trade dataframes
df_usa_grouped.rename(columns={"E_COMMODITY": "HS6", "STATE_NAME": "location", "ALL_VAL_MO": "trade_value", "YEAR": "year"}, inplace=True)

## 2. Downloading International Trade Data (BACI)

Load and clean international trade data from BACI (CEPII). This is the same source used by the Observatory of Economic Complexity (OEC).

### BACI from CEPII

The original publishers from BACI are CEPII, so we consider the database downloaded from their site (https://www.cepii.fr/CEPII/en/bdd_modele/bdd_modele_item.asp?id=37).

In [14]:
df_baci = pd.read_csv(DATASETS_DIR + "BACI/BACI_HS22_Y2023_V202501.csv", names= ["year","exporter","importer","product","value", "quantity"], skiprows=1)
df_baci.head()

,year,exporter,importer,product,value,quantity
0,2023,4,20,200290,8.01,5.83
1,2023,4,31,252620,9.81,77.15
2,2023,4,31,680221,33.79,76.66
3,2023,4,31,761510,6.60,2.37
4,2023,4,31,860900,1.50,2.20


In [15]:
df_baci.shape

(11232739, 6)

In [16]:
# The zip downloaded from CEPII includes the country codes that we read now 
df_country_codes=pd.read_csv(DATASETS_DIR + "BACI/country_codes_V202501.csv")
df_country_codes.head()

,country_code,country_name,country_iso2,country_iso3
0,4,Afghanistan,AF,AFG
1,8,Albania,AL,ALB
2,12,Algeria,DZ,DZA
3,16,American Samoa,AS,ASM
4,20,Andorra,AD,AND


In [17]:
# Merge trade database with country names
df_baci = df_baci.merge(df_country_codes, how="left", left_on="exporter", right_on="country_code")
df_baci.head()

,year,exporter,importer,product,value,quantity,country_code,country_name,country_iso2,country_iso3
0,2023,4,20,200290,8.01,5.83,4,Afghanistan,AF,AFG
1,2023,4,31,252620,9.81,77.15,4,Afghanistan,AF,AFG
2,2023,4,31,680221,33.79,76.66,4,Afghanistan,AF,AFG
3,2023,4,31,761510,6.60,2.37,4,Afghanistan,AF,AFG
4,2023,4,31,860900,1.50,2.20,4,Afghanistan,AF,AFG


In [45]:
#Quick check: total exports from USA according to BACI vs Census

print("BACI:" ,df_baci.loc[(df_baci.country_name == "USA")]["value"].sum())
print("Census:" ,df_usa_grouped["trade_value"].sum()/1000)

(df_baci.loc[(df_baci.country_name == "USA")]["value"].sum()) / (df_usa_grouped["trade_value"].sum()/1000)

BACI: 1895701752.2549996
Census: 1925839874.535


0.9843506603645968

## 3. Merging datasets

Align HS codes (2022 version) and merge U.S. state-level exports with BACI country-level data to form a single trade dataset.

In [19]:
len(df_baci["product"].unique())

5606

In [20]:
len(df_usa_grouped["HS6"].unique())

5533

In [21]:
# Group data by year
df_baci_grouped = df_baci.groupby(["country_name", "product", "year"])["value"].sum().to_frame().reset_index()
df_baci_grouped.head()

,country_name,product,year,value
0,Afghanistan,10121,2023,0.00
1,Afghanistan,10221,2023,15.78
2,Afghanistan,10619,2023,6.18
3,Afghanistan,10641,2023,126.00
4,Afghanistan,20120,2023,1.00


In [22]:
# Homogenize column names
df_baci_grouped.rename(columns={"country_name":"location", "product": "HS6","value":"trade_value"}, inplace=True)

In [23]:
# We are taking HS codes as objects of 6 characters length
df_baci_grouped['HS6'] = df_baci_grouped["HS6"].apply(lambda x: f'{x:06}')

In [25]:
 # export_value is in thousands of dollars, multiply by 1000
df_baci_grouped['trade_value'] = 1000 * df_baci_grouped['trade_value']
df_baci_grouped.head()

,location,HS6,year,trade_value
0,Afghanistan,010121,2023,2.00
1,Afghanistan,010221,2023,15775.00
2,Afghanistan,010619,2023,6178.00
3,Afghanistan,010641,2023,126005.00
4,Afghanistan,020120,2023,1000.00


In [26]:
# Export baci data grouped
df_baci_grouped.to_parquet('datasets/df_trade_baci.parquet',engine='fastparquet')

In [27]:
# Concatenate US database with BACI
df_complete = pd.concat([df_usa_grouped, df_baci_grouped])
df_complete.head()

,HS6,location,trade_value,year
0,010121,Alabama,5440.00,NaN
1,010121,Arizona,186551.00,NaN
2,010121,Arkansas,4583.00,NaN
3,010121,California,21298413.00,NaN
4,010121,Colorado,66820.00,NaN


In [30]:
# we'll store everything with descriptions
df_product_codes=pd.read_excel(DATASETS_DIR + "BACI/HSCodeandDescription.xlsx", sheet_name="HS22")
df_product_codes = df_product_codes.loc[df_product_codes["Level"]== 6]
df_product_codes.head()

,Classification,Code,Description,Parent Code,Level,IsBasicLevel
2,H6,010121,"Horses; live, pure-bred breeding animals",0101,6,1
3,H6,010129,"Horses; live, other than pure-bred breeding an...",0101,6,1
4,H6,010130,Asses; live,0101,6,1
5,H6,010190,Mules and hinnies; live,0101,6,1
7,H6,010221,"Cattle; live, pure-bred breeding animals",0102,6,1


In [31]:
df_product_codes.rename(columns={"Code": "HS6", "Description": "HS6_desc"}, inplace=True)

## 4. Quality checks and fixes

In [33]:
# Which codes didn't have a description?
df_complete = df_complete.merge(df_product_codes[["HS6", "HS6_desc"]], how="left", on="HS6")
df_complete.loc[df_complete.HS6_desc.isna()]["HS6"].unique()

array(['880000', '980110', '980210', '980220', '980230', '980240',
       '980310', '980320', '987000', '988000', '271000'], dtype=object)

In [34]:
# This one I merge it with another code of aircraft, as it's only in US states and having it there would've been distortioning
df_complete.loc[df_complete["HS6"]=="880000", "HS6"] = "880230"

In [35]:
# Manually fix lacking descriptions
df_complete.loc[df_complete["HS6"]=="271000", "HS6_desc"] = "Petroleum oils and oils from bituminous minerals, not crude: preparations n.e.c. containing by weight 70% or more of petroleum oils or oils from bituminous minerals: these being the basic constituents of the preparations: waste oils"

In [44]:
# This products don't have the description, but considering their definitions, we won't be fixing them because we won't be using them as they are only US related
# df_complete.loc[df_complete["HS6"]=="980110", "HS6_desc"] = "Value of repairs or alterations of previously imported articles, repaired or altered prior to exportation from USA"
# df_complete.loc[df_complete["HS6"]=="980210", "HS6_desc"] = "Exports of commingled food products, donated for relief or charity by individuals or private agencies"
# df_complete.loc[df_complete["HS6"]=="980220", "HS6_desc"] = "Exports of medicinal and pharmaceutical products donated for relief or charity by individuals or private agencies"
# df_complete.loc[df_complete["HS6"]=="980230", "HS6_desc"] = "Exports of all wearing apparel donated for relief or charity by individuals or private agencies"
# df_complete.loc[df_complete["HS6"]=="980240", "HS6_desc"] = "Exports of articles donated for relief or charity by individuals or private agencies"
# df_complete.loc[df_complete["HS6"]=="980310", "HS6_desc"] = "Exports of military wearing apparel of all types and materials"
# df_complete.loc[df_complete["HS6"]=="980320", "HS6_desc"] = "Exports of military equipment not identified by kind"
# df_complete.loc[df_complete["HS6"]=="987000", "HS6_desc"] = "Items imported from Canada and returned to Canada, exhibits for Canadian public museums and institutions"
# df_complete.loc[df_complete["HS6"]=="988000", "HS6_desc"] = "Low value export shipments, Canadian estimated late receipts"

In [36]:
#We are deleting this HS that appear only in US states that were related to donated for relief or charity, military equipment, etc.
exclude_hs = ["980110", "980210", "980220", "980230", "980240", "980310", "980320", "987000", "988000"]
df_complete = df_complete.loc[~df_complete.HS6.isin(exclude_hs)]
df_complete.shape

(717409, 5)

In [37]:
#We are deleting these HS due to discrepancies with the total of USA.
exclude_hs = ["711810", "711890"]
df_complete = df_complete.loc[~df_complete.HS6.isin(exclude_hs)]
df_complete.shape

(717196, 5)

In [38]:
df_complete['HS6'] = df_complete['HS6'].replace({
    "271012": "271000",
    "271019": "271000",
    "271020": "271000",
    "271091": "271000",
    "271099": "271000"
})

In [39]:
# Finally, export complete database
df_complete.to_parquet('datasets/df_trade_complete.parquet',engine='fastparquet')